In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.optimize import minimize, differential_evolution
from chisq import EclipseFit
from etv_plots import nice_units, pretty_print, Plotter, ecl_time_to_etv, lsq_fit

In [ ]:
fit = EclipseFit('5095')

In [ ]:
x = np.array([18.6108497, 66.8620090, np.radians(85.571), 0.486, np.radians(108.2), 
              239.49,  95.923, 3.762e-02, -4.031e-02, np.radians(83.46), np.radians(0.341), 
              1.0887, 1.0372, 4.58337407e-03, 0.0, 4.515, 77.5])

In [ ]:
def get_i2_Omega2(im, g1, i1, omega1):
    n1 = np.where(np.sin(omega1 - g1) > 0, omega1 - g1 + np.pi, np.pi + omega1 - g1)
    i2 = np.arccos(np.sin(im)*np.sin(i1)*np.cos(n1) + np.cos(im)*np.cos(i1))
    #i2 = np.where(np.sin(omega1 - g1) > 0, np.pi - i2, i2)
    Omega2 = np.arccos((np.cos(im) - np.cos(i1)*np.cos(i2))/(np.sin(i1)*np.sin(i2)))
    Omega2 = np.where(np.sin(omega1 - g1) > 0, Omega2, 2*np.pi - Omega2)
    return i2, Omega2

def im(i1, i2, Omega2):
    return np.arccos(np.cos(i1)*np.cos(i2) + np.sin(i1)*np.sin(i2)*np.cos(Omega2))

def n1(i1, i2, Omega2):
    n1 = np.arccos((np.cos(i2) - np.cos(i1)*np.cos(im(i1, i2, Omega2)))/(np.sin(i1)*np.sin(im(i1, i2, Omega2))))
    return np.where(np.sin(Omega2) < 0, n1, np.pi - n1)

def g1(i1, i2, Omega2, omega1):
    g1 = omega1 - n1(i1, i2, Omega2)
    return np.where(np.sin(Omega2) < 0, g1 + np.pi, g1)

# tests
i1, omega1 = x[2], x[4]
print(np.degrees(im(i1, *get_i2_Omega2(np.radians(60.0), np.radians(140.0), i1, omega1))))
print(np.degrees(g1(i1, *get_i2_Omega2(np.radians(60.0), np.radians(140.0), i1, omega1), omega1)))
print(np.degrees(g1(np.radians(90), np.radians(60), np.radians(-90), np.radians(90))))
np.degrees(get_i2_Omega2(np.radians(120.0), np.radians(180.0), np.radians(90), np.radians(90)))
np.degrees(get_i2_Omega2((*np.radians((11.2, 300, 86.0, 200+180)))))

In [ ]:
def dbdt_im_g1(angles):
    im, g1 = angles
    y = x.copy()
    i2, Omega2 = get_i2_Omega2(im, g1, x[2], x[4])
    y[9], y[10] = i2, Omega2
    ecl_model, rv_model = fit.get_residuals(y, safe=False)
    dbdt, b0 = fit.impact_regression(ecl_model)
    return dbdt['A']

def dbdt_i2_Omega2(angles):
    y = x.copy()
    y[9], y[10] = angles
    ecl_model, rv_model = fit.get_residuals(y, safe=False)
    dbdt, b0 = fit.impact_regression(ecl_model)
    return dbdt['A'] 

def dbdt_im_Omega2(angles):
    im, Omega2 = angles
    y = x.copy()
    i2 = get_i2(im, x[2], Omega2)
    y[9], y[10] = i2, Omega2
    ecl_model, rv_model = fit.get_residuals(y, safe=False)
    dbdt, b0 = fit.impact_regression(ecl_model)
    return dbdt['A'] 

In [ ]:
#plt.plot(np.degrees(np.linspace(0, 2*np.pi, 100)), [dbdt_im_g1(np.radians((90, g1))) for g1 in np.linspace(0, 360, 100)]) 

In [ ]:
N = 5000
phi = (1 + np.sqrt(5))/2
ims = np.radians(180) * np.sqrt(np.arange(N) + 1/2)/np.sqrt(N + 1/2)
g1s = 2*np.pi*np.arange(N)/phi**2
fig, ax = plt.subplots(subplot_kw=dict(projection='polar'), figsize=(10,10))
ax.scatter(g1s, np.degrees(ims), marker='.')

In [ ]:
N = 5000
phi = (1 + np.sqrt(5))/2
i2s = np.radians(180) * np.sqrt(np.arange(N) + 1/2)/np.sqrt(N + 1/2)
Omega2s = 2*np.pi*np.arange(N)/phi**2
#fig, ax = plt.subplots(subplot_kw=dict(projection='polar'), figsize=(10,10))
#ax.scatter(Omega2s, i2s, marker='.')

In [ ]:
import multiprocessing

def parallel_apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """
    Like numpy.apply_along_axis(), but takes advantage of multiple
    cores.
    """        
    # Effective axis where apply_along_axis() will be applied by each
    # worker (any non-zero axis number would work, so as to allow the use
    # of `np.array_split()`, which is only done on axis 0):
    effective_axis = 1 if axis == 0 else axis
    if effective_axis != axis:
        arr = arr.swapaxes(axis, effective_axis)

    # Chunks for the mapping (only a few chunks):
    chunks = [(func1d, effective_axis, sub_arr, args, kwargs)
              for sub_arr in np.array_split(arr, multiprocessing.cpu_count())]

    pool = multiprocessing.Pool()
    individual_results = pool.map(unpacking_apply_along_axis, chunks)
    # Freeing the workers:
    pool.close()
    pool.join()

    return np.concatenate(individual_results)

def unpacking_apply_along_axis(arg):
    """
    Like numpy.apply_along_axis(), but and with arguments in a tuple
    instead.

    This function is useful with multiprocessing.Pool().map(): (1)
    map() only handles functions that take a single argument, and (2)
    this function can generally be imported from a module, as required
    by map().
    """
    (func1d, axis, arr, args, kwargs) = arg
    return np.apply_along_axis(func1d, axis, arr, *args, **kwargs)

In [ ]:
try:
    dbdt_im_g1_arr = np.load('im_g1_fit_out/im_g1_circle.npy')
except:
    dbdt_im_g1_arr = parallel_apply_along_axis(dbdt_im_g1, 0, np.vstack((ims, g1s)))

In [ ]:
#dbdt_i2_Omega2_arr = parallel_apply_along_axis(dbdt_i2_Omega2, 0, np.vstack((i2s, Omega2s)))

In [ ]:
#dbdt_im_Omega2_arr = parallel_apply_along_axis(dbdt_im_Omega2, 0, np.vstack((ims, Omega2s)))

In [ ]:
#dbdt_im_g1_zoom_arr = parallel_apply_along_axis(dbdt_im_g1, 0, np.vstack((ims, g1s)))

In [ ]:
# planet phase shouldn't matter, so I don't compute it correctly
tp = 158.9
# Orbital solution from Borkovits et al. 2016, assuming equal mass binary
borko = [18.61196, 66.8735, np.radians(86), 0.05, np.radians(270),
         236.26, tp, 0.071*np.cos(np.radians(324)), 0.071*np.sin(np.radians(324)), np.radians(73), np.radians(39),
         1.0, 1.0, 0.018, 0.0, 0.0, 0.0]

# Orbital solution from Getley et al. 2017
getley = [18.61196, 66.8735, np.radians(80), 0.246, np.radians(22.82),
          237.70817, tp, 0.0604*np.cos(np.radians(27.67)), 0.0604*np.sin(np.radians(27.67)), np.radians(105.92), np.radians(64.19 - 305.54),
          1.21, 0.51, 7.698/1047.5, 0.0, 0.0, 0.0]

print(np.degrees(im(borko[2], borko[9], borko[10])))
print(np.degrees(g1(borko[2], borko[9], borko[10], borko[4])))
print(np.degrees(im(getley[2], getley[9], getley[10])))
print(np.degrees(g1(getley[2], getley[9], getley[10], getley[4])))
print(np.degrees(im(x[2], x[9], x[10])))
print(np.degrees(g1(x[2], x[9], x[10], x[4])))

In [ ]:
from scipy.signal import detrend

def combine_params(plan_params, x, angles):
    im, g1 = angles
    y = x.copy()
    i2, Omega2 = get_i2_Omega2(im, g1, x[2], x[4])
    y[9], y[10] = i2, Omega2
    y[5:9] = plan_params[:-1]
    y[13] = plan_params[-1]
    return y

def planet_model(plan_params, x, angles):
    y = combine_params(plan_params, x, angles)
    ecl_model, rv_model = fit.get_residuals(y, safe=True, tFin=1500)
    return ecl_model, rv_model

def planet_chisq(plan_params, x, angles):
    ecl_model, rv_model = planet_model(plan_params, x, angles)
    return ((detrend(ecl_model['A']['res'].dropna())/ecl_model['A']['data_err'].dropna())**2).sum()

In [ ]:
def reduced_fit(angles):
    result = differential_evolution(planet_chisq, 
                      args=(x, angles), popsize=5, workers=8,
                      bounds=[(220, 260), (0, 250), (-0.2, 0.2), (-0.2, 0.2), (0., 20/1000.)])
    result['angles'] = angles
    print(result)
    print('\n')
    return result

In [ ]:
import pickle
reduced_fit_results = []
for i in range(5):
    infile = open('im_g1_fit_out/im_g1_fit_{}.data'.format(i), 'rb')
    reduced_fit_results.extend(pickle.load(infile))
    infile.close()

for result in reduced_fit_results:
    if not result['success']:
        print(result)

In [ ]:
red_ims = np.asarray([result['angles'][0] for result in reduced_fit_results])
red_g1s = np.asarray([result['angles'][1] for result in reduced_fit_results])
chisqs = np.asarray([result['fun'] if result['success'] else np.nan for result in reduced_fit_results])

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='polar'), figsize=(10,10))
img = ax.scatter(red_g1s, np.degrees(red_ims), c=chisqs, s=20, alpha=0.8,
                cmap=plt.get_cmap('viridis'), #norm=colors.LogNorm(vmin=60, vmax=1000))
                 vmin=60, vmax=200)

ax.plot(np.linspace(0, 2*np.pi, 100), 90.0*np.ones(100), color='k', linestyle='--')
ax.grid(False)
ax.set_yticklabels([])
ax.set_ylim([0, 180])
cbar = plt.colorbar(img, aspect=10, shrink=0.4)
cbar.set_label('$\chi^2_{ETV}$', rotation=270, fontsize='x-large', labelpad=20)
#plt.savefig('figures/im_g1_fit_constraints.png', dpi='figure')
#plt.savefig('figures/chisq_etv.eps')

In [ ]:
for i, result in enumerate(reduced_fit_results):
    if result['angles'][0] > np.radians(90) and result['fun'] < 120:
        print(i)
        print(np.degrees(result['angles'][0]), np.fmod(np.degrees(result['angles'][1]), 360))
        print(result['fun'])
        print('')

In [ ]:
plot = Plotter('5095')

result = reduced_fit_results[2300]

plot.etv_residuals(combine_params(result['x'], x, result['angles']), 'A', phased=True, linearize=True)
print(np.fmod(np.degrees(result['angles']), 360))
print(result['x'])
print(result['fun'])

ecl_model, rv_model = fit.get_residuals(combine_params(result['x'], x, result['angles']))
(fit.impact_regression(ecl_model)[0]['A'] - fit.dbdt_data['A'])/fit.dbdt_err['A']

In [ ]:
def dbdt_fit(i):
    result = reduced_fit_results[i[0]]
    ecl_model, rv_model = fit.get_residuals(combine_params(result['x'], x, result['angles']))
    return fit.impact_regression(ecl_model)[0]['A']

In [ ]:
try:
    dbdt_fit_arr = np.load('im_g1_fit_out/im_g1_fit.npy')
except:
    dbdt_fit_arr = parallel_apply_along_axis(dbdt_fit, 1, np.arange(5000)[:,np.newaxis])

In [ ]:
sigmas = np.abs((dbdt_fit_arr - 3.99e-07)/1.69e-7)
fig, ax = plt.subplots(subplot_kw=dict(projection='polar'), figsize=(10,10))
img = ax.scatter(g1s, np.degrees(ims), c=sigmas, s=20, alpha=0.8,
                cmap=plt.get_cmap('viridis'), vmin=0, vmax=20)
#ax.scatter([g1(borko[2], borko[9], borko[10], borko[4])], [np.degrees(im(borko[2], borko[9], borko[10]))], color='orange')
#ax.scatter([g1(getley[2], getley[9], getley[10], getley[4])], [np.degrees(im(getley[2], getley[9], getley[10]))], color='k')
ax.scatter([g1(x[2], x[9], x[10], x[4])], [np.degrees(im(x[2], x[9], x[10]))], color='r', marker='x')
ax.plot(np.linspace(0, 2*np.pi, 100), 90.0*np.ones(100), color='k', linestyle='--')
ax.grid(False)
ax.set_yticklabels([])
ax.set_ylim([0, 180])
cbar = plt.colorbar(img, aspect=10, shrink=0.4)
cbar.set_label(r'$\chi^2_{dbdt}$', rotation=270, fontsize='x-large', labelpad=20)
#plt.savefig('figures/dbdt_plot.eps')
#plt.savefig('figures/tennis ball.png', dpi='figure')

In [ ]:
fig, axs = plt.subplots(1, 2, subplot_kw=dict(projection='polar'), figsize=(16,12))

img = axs[0].scatter(red_g1s, np.degrees(red_ims), c=chisqs, s=20, alpha=0.8,
                cmap=plt.get_cmap('viridis'), vmin=60, vmax=200)
cbar0 = fig.colorbar(img, ax=axs[0], aspect=10, shrink=0.6, orientation='horizontal', pad=0.05)
cbar0.set_label('$\chi^2_{ETV}$', fontsize='x-large', labelpad=20)
img = axs[1].scatter(g1s, np.degrees(ims), c=sigmas, s=20, alpha=0.8,
                cmap=plt.get_cmap('viridis'), vmin=0, vmax=20)
cbar1 = fig.colorbar(img, ax=axs[1], aspect=10, shrink=0.6, orientation='horizontal', pad=0.05)
cbar1.set_label(r'$\chi^2_{dbdt}$', fontsize='x-large', labelpad=20)

for i in (0,1):
    axs[i].plot(np.linspace(0, 2*np.pi, 100), 90.0*np.ones(100), color='k', linestyle='--')
    axs[i].grid(False)
    axs[i].set_yticklabels([])
    axs[i].set_xticklabels([])
    axs[i].set_ylim([0, 180])
    axs[i].scatter([g1(x[2], x[9], x[10], x[4])], [np.degrees(im(x[2], x[9], x[10]))], color='r', marker='x')

axs[1].annotate('stationary polar orbit', xy=(np.pi/2, 90), xycoords='data',
                xytext=(np.pi/2, 130),
                arrowprops=dict(arrowstyle='-|>', fc='k', lw=1.5),
                horizontalalignment='center', verticalalignment='top', fontsize=16)
axs[1].annotate('stationary polar orbit', xy=(3*np.pi/2, 90), xycoords='data',
                xytext=(3*np.pi/2, 130),
                arrowprops=dict(arrowstyle='-|>', fc='k', lw=1.5),
                horizontalalignment='center', verticalalignment='bottom', fontsize=16)

axs[0].annotate('our solution', xy=(np.radians(30), 5),
                xytext=(np.radians(60), 60),
                arrowprops=dict(arrowstyle='-|>', fc='k', ec='k', lw=1.5),
                horizontalalignment='center', verticalalignment='center', fontsize=16)

plt.tight_layout()
#plt.savefig('figures/combined constraint.png', dpi='figure')
#plt.savefig('figures/combined_constraint.eps')

In [ ]:
import pickle
reduced_fit_results = []
for i in range(5):
    infile = open('im_g1_fit_out/im_g1_fit_{}_dense.data'.format(i), 'rb')
    reduced_fit_results.extend(pickle.load(infile))
    infile.close()

for result in reduced_fit_results:
    if not result['success']:
        print(result)

In [ ]:
red_ims = np.asarray([result['angles'][0] for result in reduced_fit_results])
red_g1s = np.asarray([result['angles'][1] for result in reduced_fit_results])
chisqs = np.asarray([result['fun'] if result['success'] else np.nan for result in reduced_fit_results])

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='polar'), figsize=(10,10))
img = ax.scatter(red_g1s, np.degrees(red_ims), c=chisqs, s=20, alpha=0.8,
                cmap=plt.get_cmap('viridis'), #norm=colors.LogNorm(vmin=60, vmax=1000))
                 vmin=60, vmax=200)

#ax.plot(np.linspace(0, 2*np.pi, 100), 90.0*np.ones(100), color='k', linestyle='--')
ax.grid(False)
ax.set_yticklabels([])
ax.set_ylim([0, 90])
cbar = plt.colorbar(img, aspect=10, shrink=0.4)
cbar.set_label('$\chi^2_{ETV}$', rotation=270, fontsize='x-large', labelpad=20)
#plt.savefig('figures/im_g1_fit_constraints.png', dpi='figure')
#plt.savefig('figures/chisq_etv.eps')

In [ ]:
N = 5000
ims = np.zeros(0)
g1s = np.zeros(0)

# Create a sphere
a = 4*np.pi/N
d = np.sqrt(a)
M_theta = int(np.round(np.pi/d))
d_theta = np.pi/M_theta
d_phi = a/d_theta
for m in range(M_theta):
    theta = np.pi*(m + 0.5)/M_theta
    M_phi = int(np.round(2*np.pi*np.sin(theta)/d_phi))
    for n in range(M_phi):
        phi = 2*np.pi*n/M_phi
        ims = np.append(ims, theta)
        g1s = np.append(g1s, phi)

In [ ]:
try:
    dbdt_im_g1_sphere = np.load('im_g1_fit_out/im_g1_sphere.npy')
except:
    dbdt_im_g1_sphere = parallel_apply_along_axis(dbdt_im_g1, 0, np.vstack((ims, g1s)))

In [ ]:
%matplotlib auto
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D

xx = np.sin(ims)*np.cos(g1s)
yy = np.sin(ims)*np.sin(g1s)
zz = np.cos(ims)

#Set colours and render
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

#ax.plot_surface(
#    x, y, z,  rstride=1, cstride=1, color='c', alpha=0.3, linewidth=0)
sigmas = np.abs((dbdt_im_g1_sphere - 3.99e-07)/1.69e-7)
img = ax.scatter(xx, yy, zz, c=sigmas, s=20,
                cmap=plt.get_cmap('viridis'), norm=colors.LogNorm(vmin=sigmas.min(), vmax=sigmas.max()))

ax.scatter([0], [0], [1], c='red')
ax.scatter([0], [0], [-1], c='black')
ax.plot(np.cos(np.linspace(0, 2*np.pi, 100)), np.sin(np.linspace(0, 2*np.pi, 100)), np.zeros(100), color='k', linestyle='--')

ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])
cbar = plt.colorbar(img, aspect=10, shrink=0.4)
cbar.set_label('$\sigma$ score', rotation=270, fontsize='x-large')
#ax.set_aspect("equal")
plt.tight_layout()
plt.show()

In [ ]:
#np.save('im_g1_circle.npy', dbdt_im_g1_arr)
#np.save('im_g1_sphere.npy', dbdt_im_g1_sphere)